# 발표 소단원 리스트

6-8, 6-11, 7-1, 7-4, 7-7

# 전체 요약자료

- 급하면 최소한 이거라도 보고 오시라고!!
- 전체 내용에 대한 문항별 요약도 좋고, 자기 나름대로의 요약도 좋습니다!!
- 


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 문항별 자료 

## 5.20 시리얼 포트와 통신

In [ ]:
import serial
ser = serial.Serial('/dev/tty.usbmodem641',
                   baudrate=9600,
                   bytesize=8,
                   parity='N',
                   stopbits=1)

ser.write(b'G1 X50 Y50\r\n') #모든 입출력은 바이너리
resp = ser.readline()

- 보통 로봇에서는 ros(robot operating system)으로 로봇과 통신

컴퓨터에서 동작하는 ROS 와 8051, AVR, ARM Cortex-M 등의 MCU(Micro Controller Unit)간의 메시지 통신을 위해 rosserial 이라는 ROS의 패키지가 제공

ROS 대부분의 패키지들은 상위레벨의 소프트웨어가 중심인데 로봇을 실제로 동작하기 위해서는 하드웨어와 연결하여 하위레벨 제어가 필요해진다. 즉, 로봇의 이동 명령을 상위레벨의 소프트웨어에서 만들었다고 하더라도 결국은 모터를 제어하는 하위레벨이 필요하다는 것이다. 이때에는 실시간 제어가 가능하도록 위에서 언급한 MCU 들을 이용하게 되는데 이때의 중간 중계역할을 rosserial이 담당한다.

## 6.8 관계형 데이터베이스 작업
- 선택, 삽입, 행 삭제 작업

In [2]:
stocks = [
    ('goog', 100, 490.1),
    ('AAPL', 50, 565.75),
    ('fb', 150, 7.45),
    ('hpq', 75, 33.2)
]

In [4]:
import sqlite3
#데이터 베이스를 연결
db = sqlite3.connect('database.db')  #데이터 베이스 이름, 호스트 이름, 사용자 이름 ,암호 등 필요한 정보

In [5]:
#커서 생성
c = db.cursor()
#c.execute('drop table if exists portfolio')
c.execute('CREATE TABLE IF NOT EXISTS portfolio1 (symbol text, shares integer, price real)')
db.commit()

In [6]:
c.executemany('insert into portfolio1 values (?,?,?)', stocks)
db.commit()

In [7]:
for row in db.execute('select * from portfolio1'):
    print(row)

('goog', 100, 490.1)
('AAPL', 50, 565.75)
('fb', 150, 7.45)
('hpq', 75, 33.2)


In [8]:
min_price = 100
for row in db.execute('select * from portfolio1 where price >= ?', (min_price,)):
    print(row)

('goog', 100, 490.1)
('AAPL', 50, 565.75)


데이터 베이스 자료를 파이썬 타입에 매핑할 때 :
    
    날짜 자료는 datetime 모듈의 datetime 인스턴스타 타임스태프 사용 / 금율 자료와 같이 숫자를 저장할 때는 decimal 모듈의 Decimal 인스턴스를 사용
    
    (데이터베이스 백엔드에 따라 달라져서 관련 문서 참고)
    

SQL 구문 문자열 서식화 :
    파이썬의 서식화 연산자(%)나 .format() 메소드로 만들면 안됨(sql 주입 공격으로 인해?)

## 6.11 바이너리 배열 구조체 읽고 쓰기
- 바이너리 배열 구조를 파이썬 튜플로 읽거나 쓰기 - struct 모듈

In [9]:
from struct import Struct

def write_records(records, format, f): #format : 포맷문자열
    '''
    일련의 튜플을 구조체의 바이너리 파일에 기록
    '''
    record_struct = Struct(format)
    for r in records:
        f.write(record_struct.pack(*r))

In [ ]:
if __name__=='__main__':
    records = [(1, 2.3, 4.5),
              (6, 7.8, 9.0),
              (12, 13.4, 56.7)]
    with open('data.b', 'wb') as f:
        write_records(records, '<idd', f) #"idd"이면 32비트 정수

idd : 32비트 정수, 64비트 부동 소수, 32비트 부동 소수

< : 바이트 순서 (little endian)

'>' : big endian

! : 네트워크 바이트 순서

In [10]:
#파일을 조각조각 읽어 들임
def read_records(format, f):
    record_struct = Struct(format)
    chunks = iter(lambda: f.read(record_struct.size), b'')
    return (record_struct.unpack(chunk) for chunk in chunks)

In [ ]:
if __name__ == '__main__':
    with open('data.b', 'rb') as f:
        for rec in read_records('<idd', f):
            # rec 처리

In [11]:
#파일을 한번에 읽어 들이고 추후 여러 조각으로 변환할 때
def unpack_records(format, data):
    record_struct = Struct(format)
    return (record_struct.unpack_from(data, offset)
           for offset in range(0, len(data), record_struct.size))

In [ ]:
with open('data.b', 'rb') as f:
    data=f.read()
for rec in unpack_records('<idd', data):
    #rec 처리

- struct 모듈 = 바이너리 데이터 인코딩, 디코딩

record_struct = Struct('<idd') 로 새로운 구조체 선언.

In [12]:
record_struct = Struct('<idd')
record_struct.size #구조체의 크기를 바이트로 나타내고 있음

20

In [13]:
record_struct.pack(1, 2.0, 3.0)

b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@'

In [14]:
record_struct.unpack(_)

(1, 2.0, 3.0)

## 7.1 매개변수 개수에 구애 받지 않는 함수 작성
 - '*' 인자 사용

In [15]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))# Sample use
avg(1, 2)          # 1.5
avg(1, 2, 3, 4)    # 2.5

1.5

2.5

-키워드 매개변수 수에 제한이 없는 함수를 작성하려면 **로 시작하는 인자 사용

In [16]:
import html
def make_element(name, value, **attrs): #attrs : 절달받은 키워드 매개변수를 저장하는 딕셔너리
    keyvals = [' %s="%s"' % item for item in attrs.items()]    
    attr_str = ''.join(keyvals)    
    element = '<{name}{attrs}>{value}</{name}>'.format(
        name=name,
        attrs=attr_str,
        value=html.escape(value))
    return element

# '<item size="large" quantity="6">Albatross</item>' 생성
make_element('item', 'Albatross', size='large', quantity=6)

# '<p>&lt;spam&gt;</p>' 생성
make_element('p', '<spam>')

'<item size="large" quantity="6">Albatross</item>'

'<p>&lt;spam&gt;</p>'

- 위치 매개변수와 키워드 매개변수를 동시에 받는 함수를 작성하려면 ' * ' 와 **를 함께 사용

In [17]:
def anyargs(*args, **kwargs):
    print(args)      # 위치 매개변수는 튜플 args에 들어감
    print(kwargs)    # 키워드 매개변수는 딕셔너리 kwargs에 들어감

## 7.4 함수에서 여러 값을 반환
- 튜플 사용

In [18]:
def myfunc():
    return 1, 2, 3

a, b, c = myfunc()
a
b
c

1

2

3

In [19]:
a = (1, 2)
a

(1, 2)

In [20]:
b = 1, 2 #실제로 튜플을 생성하는 것은 쉼표 (괄화 아님)
b

(1, 2)

In [21]:
x = myfunc()
x

(1, 2, 3)

## 7.7 이름 없는 함수에서 변수 고정

In [22]:
x = 10
a = lambda y : x + y
x = 20
b = lambda y : x + y

In [23]:
a(10)
b(10)

30

30

- lambda에서 사용한 x 값은 실행 시간에 달라지는 변수 -> 람다 표현식의 x 값은 그 함수를 실행할 때의 값이 됨

In [24]:
x = 15
a(10)

25

In [25]:
x = 3
a(10)

13

특정 값을 고정하고 싶으면 그 값을 기본 값으로 지정함

In [26]:
x = 10
a = lambda y, x=x : x + y
x = 20
b = lambda y, x=x: x + y
a(10)
b(10)

20

30

In [27]:
funcs = [lambda i: i+n for n in range(5)]
for f in funcs: 
    print(f(0))

4
4
4
4
4


람다 함수는 가장 마지막 값 사용

In [28]:
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

0
1
2
3
4
